In [25]:
import pandas as pd

In [26]:
from google.colab import drive

drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [27]:
log_spec = pd.read_csv("/content/gdrive/MyDrive/빅콘 군집분석/log_spec_cluster.csv")

In [28]:
log_spec.income_type.value_counts()

EARNEDINCOME       213220
OTHERINCOME         54147
PRIVATEBUSINESS     42138
FREELANCER          32714
EARNEDINCOME2       20100
PRACTITIONER         8098
Name: income_type, dtype: int64

결측치 및 이상치 처리

1. purpose 영한 결합

In [29]:
log_spec.loc[log_spec.purpose == "LIVING","purpose"] = "생활비"
log_spec.loc[log_spec.purpose == "SWITCHLOAN","purpose"] = "대환대출"
log_spec.loc[log_spec.purpose == "BUSINESS","purpose"] = "사업자금"
log_spec.loc[log_spec.purpose == "HOUSEDEPOSIT","purpose"] = "전월세보증금"
log_spec.loc[log_spec.purpose == "ETC","purpose"] = "기타"
log_spec.loc[log_spec.purpose == "BUYHOUSE","purpose"] = "주택구입"
log_spec.loc[log_spec.purpose == "INVEST","purpose"] = "투자"
log_spec.loc[log_spec.purpose == "BUYCAR","purpose"] = "자동차구입"

2. personal_rehabilitation

In [30]:
#개인회생자가 아닌데 납입완료(이상치) -> drop
no_user1=log_spec.loc[(log_spec['personal_rehabilitation_yn']==0) & (log_spec['personal_rehabilitation_complete_yn']==1),'application_id']
no_user1=log_spec[(log_spec['personal_rehabilitation_yn']==0) & (log_spec['personal_rehabilitation_complete_yn']==1)].index
log_spec.drop(no_user1,inplace=True)

#개인회생자가 아닌 사람 -> 1
condition2 = log_spec['personal_rehabilitation_yn']==0
log_spec.loc[condition2, 'personal_rehabilitation_complete_yn']= '1'

#개인회생자중 납입중 -> 2
condition2 = log_spec['personal_rehabilitation_complete_yn']==0
log_spec.loc[condition2, 'personal_rehabilitation_complete_yn']= '2'

#개인회생자중 납입완료 -> 3
condition3 = log_spec['personal_rehabilitation_complete_yn']==1
log_spec.loc[condition3, 'personal_rehabilitation_complete_yn']= '3'

#개인회생자 버튼 건너뛰기(NULL값) -> 0
log_spec['personal_rehabilitation_complete_yn'] = log_spec['personal_rehabilitation_complete_yn'].fillna('0')

log_spec['personal_rehabilitation_complete_yn'].value_counts()

1    258046
0    106329
2      5552
3       508
Name: personal_rehabilitation_complete_yn, dtype: int64

3. existing_loan_amt

In [31]:
# loan_cnt = 1 인데 loan_amt = nan 인 애들 -> 금융사에서 제공하지 않은 정보라고 보고 1의 평균값으로 대체
loan_amt_null = log_spec.existing_loan_amt.isna() == True
log_spec.loc[loan_amt_null,'existing_loan_amt'] = 0

# loan_cnt가 nan인 애들 -> 마이데이터에
loan_cond = (log_spec['existing_loan_cnt'].isna() == True)
log_spec.loc[loan_cond,'existing_loan_cnt'] = 0
log_spec.loc[loan_cond, 'existing_loan_amt'] = 0

4. company_enter

In [32]:
log_spec['company_enter_month'] = log_spec['company_enter_month'].fillna(-1)
log_spec['company_enter_month'] = log_spec['company_enter_month'].astype(int).astype(str)

In [33]:
log_spec['company_enter_month'] = log_spec['company_enter_month'].apply(lambda x : x[:-2]  if len(x)>=8 else x)

In [34]:
import numpy as np

In [35]:
log_spec['company_enter_month'] = log_spec['company_enter_month'].replace('-1',np.nan)

In [36]:
log_spec['company_enter_month']= pd.to_datetime(log_spec['company_enter_month'],format='%Y%m')

In [37]:
# insert 타임에서의 연차 계산
def diff_month2(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [38]:
log_spec['insert_time'] = pd.to_datetime(log_spec['insert_time'])

In [39]:
log_spec['company_month'] = log_spec.apply(lambda x: diff_month2(x.insert_time, x.company_enter_month), axis=1)

In [40]:
log_spec['company_month']=log_spec['company_month'].fillna(0)

나머지 결측치 - MICE


In [41]:
log_spec.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370435 entries, 0 to 370435
Data columns (total 30 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   Unnamed: 0                           370435 non-null  int64         
 1   user_id                              370435 non-null  int64         
 2   event_CompleteIDCertification        370435 non-null  float64       
 3   event_EndLoanApply                   370435 non-null  float64       
 4   event_GetCreditInfo                  370435 non-null  float64       
 5   event_Login                          370435 non-null  float64       
 6   event_OpenApp                        370435 non-null  float64       
 7   event_SignUp                         370435 non-null  float64       
 8   event_StartLoanApply                 370435 non-null  float64       
 9   event_UseDSRCalc                     370435 non-null  float64       
 

In [42]:
log_spec = log_spec.drop(['Unnamed: 0','insert_time','personal_rehabilitation_yn','company_enter_month'], axis = 1)

In [43]:
log_spec = pd.get_dummies(log_spec)

In [44]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [45]:
df = log_spec.drop(['user_id','application_id'], axis = 1)

In [46]:
imputer_mice = IterativeImputer(random_state = 0)
df2 = imputer_mice.fit_transform(df) #mice로 결측치 대체하여 df2에 저장
df2 = pd.DataFrame(df2, columns = df.columns)

In [47]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370435 entries, 0 to 370434
Data columns (total 45 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   event_CompleteIDCertification          370435 non-null  float64
 1   event_EndLoanApply                     370435 non-null  float64
 2   event_GetCreditInfo                    370435 non-null  float64
 3   event_Login                            370435 non-null  float64
 4   event_OpenApp                          370435 non-null  float64
 5   event_SignUp                           370435 non-null  float64
 6   event_StartLoanApply                   370435 non-null  float64
 7   event_UseDSRCalc                       370435 non-null  float64
 8   event_UseLoanManage                    370435 non-null  float64
 9   event_UsePrepayCalc                    370435 non-null  float64
 10  event_ViewLoanApplyIntro               370435 non-null  

In [48]:
df2.to_csv('cluster_type2.csv')